In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
import json
import torch
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from helpers.utils import get_metrics, set_matplotlib_params
from datasets.moldataset import MolDataHuman
from networks.nonlinearnet_aihuman import NonLinearNetDefer, optimization_loop

set_matplotlib_params()
warnings.filterwarnings('ignore')
seed = 12
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(seed)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
drd2_train = pd.read_csv("datasets/drd2_train_ECFP_counts.csv")
drd2_train_undersampled = pd.read_csv("datasets/drd2_train_undersampled_ECFP_counts.csv")
drd2_test = pd.read_csv("datasets/drd2_test_ECFP_counts.csv")
d = 2048

print(f"Train size: {drd2_train.shape}")
print(f"Train undersampled size: {drd2_train_undersampled.shape}")
print(f"Test size: {drd2_test.shape}")

Train size: (21302, 2052)
Train undersampled size: (2420, 2053)
Test size: (5404, 2052)


In [4]:
drd2_train_undersampled["activity_y"] = drd2_train_undersampled.activity.values.tolist()
drd2_train_undersampled["activity_h"] = drd2_train_undersampled.activity.values.tolist()

drd2_test["activity_y"] = drd2_test.activity.values.tolist()
drd2_test["activity_h"] = drd2_test.activity.values.tolist()

In [5]:
train_features = drd2_train_undersampled[[f"bit{i}" for i in range(d)]].values
train_labels = drd2_train_undersampled[["activity_y", "activity_h"]].values

test_features = drd2_test[[f"bit{i}" for i in range(d)]].values
test_labels = drd2_test[["activity_y", "activity_h"]].values

X_train = torch.tensor(train_features, dtype=torch.float32)
y_train = torch.tensor(train_labels, dtype=torch.float32)

X_test = torch.tensor(test_features, dtype=torch.float32)
y_test = torch.tensor(test_labels, dtype=torch.float32)

In [6]:
# example usage:
num_features = train_features.shape[1]  # number of input features
num_epochs = 100
lr = 0.1

# create an instance of the NonLinearNetDefer
l2d_model = NonLinearNetDefer(num_features)

# define the loss function and optimizer for the l2d_model
criterion = nn.BCEWithLogitsLoss()  # use BCEWithLogitsLoss for binary classification
optimizer = optim.SGD(l2d_model.parameters(), lr=lr)

In [7]:
X = X_train

# binary labels for classifier 1 and classifier 2 (or human model)
y = y_train[:,0].unsqueeze(1)
h = y_train[:,1].unsqueeze(1)

In [8]:
optimization_loop(num_epochs, optimizer, l2d_model, X, y, h, criterion)

Epoch [10/100], Loss: 2.0738112926483154
Epoch [20/100], Loss: 2.004621982574463
Epoch [30/100], Loss: 1.9217842817306519
Epoch [40/100], Loss: 1.8494293689727783
Epoch [50/100], Loss: 1.7975194454193115
Epoch [60/100], Loss: 1.7612941265106201
Epoch [70/100], Loss: 1.7316551208496094
Epoch [80/100], Loss: 1.7079859972000122
Epoch [90/100], Loss: 1.6901898384094238
Epoch [100/100], Loss: 1.6765260696411133


In [9]:
y = y_test[:,0].unsqueeze(1)
h = y_test[:,0].unsqueeze(1) # y = h

with torch.no_grad():
    l2d_model.eval()
    combined_outputs, decision_outputs = l2d_model(X_test)
    pred_clf = (combined_outputs > 0.5).float()
    metrics = {}
    labels = [y, h]
    for i in range(2):
        metrics[f"clf_{i+1}"] = get_metrics(labels[i], pred_clf[:, i])
    boolean = (decision_outputs[:, -1] > decision_outputs[:, 0]) * (decision_outputs[:, 1] > decision_outputs[:, 0]) * 1.

    final_predictions = (boolean * pred_clf[:, 1]) + (1 - boolean) * pred_clf[:, 0]
    metrics[f"system"] = get_metrics(labels[0], final_predictions)
print (json.dumps(metrics, indent=2, default=str))
print(f"Percentage of deferral: {boolean.mean()}")

{
  "clf_1": {
    "Accuracy": 0.9015544041450777,
    "Precision": 0.9614195480412367,
    "Recall": 0.9015544041450777,
    "F1-Score": 0.9206587738165207
  },
  "clf_2": {
    "Accuracy": 0.9289415247964471,
    "Precision": 0.9665384478395359,
    "Recall": 0.9289415247964471,
    "F1-Score": 0.9405052962050526
  },
  "system": {
    "Accuracy": 0.9289415247964471,
    "Precision": 0.9665384478395359,
    "Recall": 0.9289415247964471,
    "F1-Score": 0.9405052962050526
  }
}
Percentage of deferral: 0.8995188474655151
